# Network Analysis
  > A notebook to test network analysis workflows

### Lime Data Engineering

In [31]:
## Create a file geodatabase for the Lime scooter data
import arcpy

# read in the data
limeFolder = "C:/GIS/Scooters/Lime/2023"

# delete file goeodatabase if it already exists
if arcpy.Exists(r"C:\\GIS\\Scooters\\Lime\\2023\\Analysis.gdb"):
    arcpy.management.Delete(r"C:\\GIS\\Scooters\\Lime\\2023\\Analysis.gdb")

# create file geodatabase
arcpy.management.CreateFileGDB(limeFolder, "Analysis.gdb")

<Result 'C:\\GIS\\Scooters\\Lime\\2023\\Analysis.gdb'>

In [2]:
# import packages
import pandas as pd
import os
import glob
import arcpy

# read in the data
limeFolder = "C:/GIS/Scooters/Lime/2023"

# set the workspace
arcpy.env.workspace =  "C:/GIS/Scooters/Lime/2023/Analysis.gdb"

# set overwrite to true
arcpy.env.overwriteOutput = True
        
# create the JOSN files
def make_json(limeFolder):
    # get all the files in the folder
    allFiles = glob.glob(limeFolder + "/*.csv")
    # loop through the files
    for limeFile in allFiles:
        # get name of file without .csv
        fileName = os.path.splitext(limeFile)[0]
        # read in the csv
        df = pd.read_csv(limeFile, usecols=['POLYLINE'])
        # filter out polyline null values
        df = df[df.POLYLINE.notnull()]
        # load limeDF POLYLINE field into a list
        polylineList = df['POLYLINE'].tolist()
        # remove '{"type":"FeatureCollection","features":[ from the values in the list
        polylineList = [x.replace('{"type":"FeatureCollection","features":[', '') for x in polylineList]
        # remove the last two characters from the values in the list
        polylineList = [x[:-2] for x in polylineList]
        # create empty list to hold the values
        data_result = []
        # append all the values together
        data_result.append(', '.join(polylineList))
        # add strings to the beginning and end of the data_result item 
        prefix = '{"type":"FeatureCollection","features":['
        suffix = ']}'
        data_result[0] = prefix + data_result[0] + suffix
        # write the data_result to json 
        with open(fileName + ".json", 'w') as f:
            f.write(data_result[0])

# function to make the json files into feature classes
def make_fc(limeFolder):
    # set the workspace
    arcpy.env.workspace =  "C:/GIS/Scooters/Lime/2023/Analysis.gdb"
    # get all the files in the folder
    allFiles = glob.glob(limeFolder + "/*.json")
    # loop through the files
    for limeFile in allFiles:
        # get name of file without .json
        fullfileName = (os.path.splitext(limeFile)[0]).split('\\')[-1]
        fileName = fullfileName.split('_')[1] + "_" + fullfileName.split('_')[2] + "_" + fullfileName.split('_')[3]
        # set the local variables
        in_json_file = limeFile
        # set the output feature class
        out_feature_class = fileName
        # execute JSONToFeatures
        arcpy.conversion.JSONToFeatures(in_json_file, out_feature_class)
        # delete the json file
        os.remove(limeFile)

# project fcs
def project_fcs():
    # set the workspace
    arcpy.env.workspace =  "C:/GIS/Scooters/Lime/2023/Analysis.gdb"
    # arcpy overwrite
    arcpy.env.overwriteOutput = True
    # loop through the feature classes in teh workspace geodatabase
    fcs = arcpy.ListFeatureClasses()
    # loop through the files and project them
    for fc in fcs:
        outCS = arcpy.SpatialReference(26910)
        arcpy.management.Project(fc, fc + "_proj", outCS)
    
# function to make the line density rasters
def make_density_raster(size=10, neighborhood=100):
    # set the workspace
    arcpy.env.workspace = "C:/GIS/Scooters/Lime/2023/Analysis.gdb"
    # get the feature classes with _proj 

    fcs = arcpy.ListFeatureClasses('*_proj')
    # loop through the files and run line density
    for fc in fcs:
        desc = arcpy.Describe(fc)
        # Set local variables
        inPolylineFeatures = fc
        populationField = "NONE"
        cellSize = size
        searchRadius = neighborhood
        out_raster = desc.name + "_line_density_size_"+str(size)+"_neighborhood_"+str(neighborhood) # get name of feature class
        # Execute LineDensity
        outLineDensity = arcpy.sa.LineDensity(inPolylineFeatures, populationField, cellSize,
                                    searchRadius, "SQUARE_MILES") 
        # Save the output 
        outLineDensity.save(out_raster)

In [27]:
# create the JSON files
make_json(limeFolder)

In [43]:
# make the feature classes from the JSON files
make_fc(limeFolder)

In [3]:
# project the feature classes
project_fcs()

In [49]:
# make the line density rasters from the feature classes
make_density_raster(size=10, neighborhood=100)

#### Other

In [ ]:
# Combine all the csv files into one csv
os.chdir(limeFolder)
# list csvs in folder
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
# print(all_filenames)
# combine all files in the list and add field for filename for month and year
combined_csv = pd.concat([pd.read_csv(f).assign(filename=os.path.basename(f)) for f in all_filenames ])
#export to csv
combined_csv.to_csv( limeFolder + "\\All_Lime_Trips_2023.csv", index=False, encoding='utf-8-sig')
